In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import sqlite3
import os
import numpy as np
from requests_html import HTMLSession
from requests_file import FileAdapter

session = HTMLSession()

session.mount('file://',FileAdapter())
pwd = os.getcwd().replace("\\","/")
html_obj = session.get(f'file:///{pwd}/Anue鉅亨.html')

soup = BeautifulSoup(html_obj.text,"html.parser")

In [9]:
titletime = []
id = []
content = []
  

# # 日期============================
times = soup.select('._67tN')
for time in times:
  head,mid,tail = time.find('time')['datetime'].partition('T') # partition‘T'之後分為頭部，中部，尾部三個
  titletime.append(head)
# print(len(titletime))

# 內容============================
titles = soup.select('._1xc2')
for title in titles:
  head,mid,tail = title.find('h3').text.partition(':')
  id.append(head)
  content.append(title.text)
# print(len(id))
# print(len(content))


data = {
  'date': titletime,
  'code': id,
  'content': content
}


df = pd.DataFrame(data)
# print(df)

# # 今日新闻
# newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
# df_mask = df['date']=='2022-08-27'            # 日期為今天的存檔
# positions = np.flatnonzero(df_mask)
# filtered_df=df.iloc[positions]
# print(filtered_df)


# connecting to sqlite
conn = sqlite3.connect('anuenews.db')
# cursor object
cursor = conn.cursor()
uniqueSql = '''select * from news t'''


for index, row in df.iterrows():
  try:
    cursor.execute(
        """INSERT OR IGNORE INTO news 
          (date, code, content)
          values(?,?,?)""",
          (row['date'],
          row['code'],
          row['content']),
        )
    conn.commit()
  except:
    pass

pd.read_sql(uniqueSql,conn)

,date,code,content
0,2022-08-23,中天,子公司中天(上海)生物科技有限公司與合一生技共同研發之SNS812廣效性抗新冠病毒小核酸新藥...
1,2022-08-23,合一,本公司與中天(上海)生物科技有限公司共同研發之廣效性抗新冠病毒小核酸新藥SNS812，今日向...
2,2022-08-23,通訊-KY,公告本公司董事會通過111年第二季合併財務報告
3,2022-08-23,通訊-KY,更正本公司董事會通過取得有價證券(更正本次交易總金額、簽約及付款日、最近期財務報表金額)
4,2022-08-23,太景*-KY,代子公司太景醫藥研發(北京)有限公司公告抗流感病毒新藥二期臨床試驗解盲結果
...,...,...,...
1211,2022-08-23,電動車需求旺盛 估明年營收比重提升至8%,
1212,2022-08-23,客戶進行庫存調整 下半年營收拚持平去年同期,
1213,2022-08-23,研華攜多家產業夥伴 推五大解決方案,
1214,2022-08-23,廣明旗下達明今年營運可望優去年 看好協作機器人前景,


In [3]:
# 导出为csv
import pandas as pd
import numpy as np
import sqlite3


conn2 = sqlite3.connect('anuenews.db')
uniqueSql2 = '''select * from news t'''

df2 = pd.read_sql(uniqueSql2,conn2)
df3 = pd.DataFrame(df2, columns=['code','content','date'])

# 今日新闻
newsToday = pd.Timestamp.today().strftime("%Y-%m-%d") # 格式化日期
# df_mask = df3['date']==newsToday            # 日期為今天的存檔
df_mask = df3['date']=='2022-08-25'           # 日期為今天的存檔
positions = np.flatnonzero(df_mask)
filtered_df=df3.iloc[positions]


filtered_df.to_csv("鉅亨每日新聞.csv", encoding="big5hkscs",index=False)
print('導出成功')

導出成功


In [ ]:
# 创建资料库 ====================================================================
# import sqlite3


# # connecting to sqlite
# conn = sqlite3.connect('anuenews.db')
# # cursor object
# cursor = conn.cursor()

# # Creating table
# news = """ CREATE TABLE news (
#     date DATE,
#     code TEXT,
#     content TEXT  UNIQUE, 
#     UNIQUE(content)
# ); """

# cursor.execute(news)
# ================================================================================

In [19]:
# 删除资料 ====================================================================
# conn = sqlite3.connect('anuenews.db')
# cursor = conn.cursor()
# sql_delete_query = """DELETE from news where code = '〈美股早盤〉市場靜待Fed升息線索 主要指數開低'"""
# cursor.execute(sql_delete_query)
# conn.commit()
# pd.read_sql(uniqueSql,conn)
# ==============================================================================

,date,code,content
0,2022-08-23,中天,子公司中天(上海)生物科技有限公司與合一生技共同研發之SNS812廣效性抗新冠病毒小核酸新藥...
1,2022-08-23,合一,本公司與中天(上海)生物科技有限公司共同研發之廣效性抗新冠病毒小核酸新藥SNS812，今日向...
2,2022-08-23,通訊-KY,公告本公司董事會通過111年第二季合併財務報告
3,2022-08-23,通訊-KY,更正本公司董事會通過取得有價證券(更正本次交易總金額、簽約及付款日、最近期財務報表金額)
4,2022-08-23,太景*-KY,代子公司太景醫藥研發(北京)有限公司公告抗流感病毒新藥二期臨床試驗解盲結果
...,...,...,...
1201,2022-08-23,太景*-KY,代子公司太景醫藥研發(北京)有限公司公告抗流感病毒新藥二期臨床試驗解盲結果
1202,2022-08-23,龍邦,代子公司保勝投資股份有限公司公告取得泰山普通股一年內累計達新台幣三億元
1203,2022-08-23,嘉泥,公告本公司取得黑石房地產信託iCapital離岸通道基金-A類 累積-(第三十批)
1204,2022-08-23,合富-KY,代重要子公司合富(中國)公告經董事會決議參與子公司合璽(香港)控股有限公司之現金增資。
